**A1.4C.2** - Does the system support advanced linguistic functions, including but not limited to latent semantic indexing or similar, concept relevance, question answering. If yes, please elaborate.


- **Latent Semantic Indexing (LSI)**: This natural language processing method enables the analysis of relationships between documents and their contained terms using Singular Value Decomposition (SVD), helping us identify patterns in term-concept relationships within unstructured text.

- **Concept Relevance**: Utilizing Watson Assistant, we can understand and respond to relevant concepts. Intents within Watson Assistant serve as key concepts for our chatbot, with two sentences classified under the same intent considered to be within the same context.

- **Question Answering**: Through Watson Assistant, we employ techniques such as intent classification and dialogue flows to provide comprehensive and accurate responses to queries.

# Latent Semantic Indexing (LSI)

Let's consider a small corpus of 3 documents and a vocabulary of 5 terms. 

Here's the term-document matrix:

|        | Document 1 | Document 2 | Document 3 |
|--------|------------|------------|------------|
| term1  |     1      |     0      |     1      |
| term2  |     0      |     1      |     0      |
| term3  |     1      |     1      |     0      |
| term4  |     1      |     0      |     0      |
| term5  |     0      |     1      |     1      |

Let's assume that after applying SVD, we got the following U, Σ, and V matrices:

**U matrix**:

|        | Topic 1 | Topic 2 |
|--------|---------|---------|
| term1  |   0.5   |   0.3   |
| term2  |   0.1   |   0.6   |
| term3  |   0.4   |   0.1   |
| term4  |   0.5   |   -0.1  |
| term5  |   0.2   |   0.7   |

**Σ matrix**:

|         | Topic 1 | Topic 2 |
|---------|---------|---------|
| Topic 1 |    3    |    0    |
| Topic 2 |    0    |    2    |

**V matrix**:

|            | Document 1 | Document 2 | Document 3 |
|------------|------------|------------|------------|
| Topic 1    |     0.6    |     0.2    |     0.5    |
| Topic 2    |     0.3    |     0.6    |     0.4    |

The U matrix represents the relationships between terms and topics. For instance, term1 has a strong relationship with Topic 1 (0.5) and a weaker relationship with Topic 2 (0.3). 

The Σ matrix contains the singular values, which correspond to the "strength" or "importance" of each topic. In this case, Topic 1 is more important (with a singular value of 3) than Topic 2 (with a singular value of 2). 

The V matrix represents the relationships between documents and topics. For example, Document 1 has a strong relationship with Topic 1 (0.6) and a weaker relationship with Topic 2 (0.3).

These matrices help us to reduce the dimensionality of our original term-document space, capture latent semantic relationships between terms and documents, and improve the performance of information retrieval tasks by allowing us to match documents and queries based on semantic similarity rather than exact term matching. 

Please note: This example is very simplified and doesn't necessarily reflect the actual values you'd get from performing SVD on a real term-document matrix, but it hopefully gives you a good idea of what each matrix represents. In practice, these matrices would be obtained through numerical methods and the values would be less clean and easy to interpret.

**LSI vs LDA**

In Latent Semantic Indexing (LSI) and similar matrix factorization methods like Non-negative Matrix Factorization (NMF), a word or a document is represented as a weighted sum of all topics, and the weights are generally non-zero for all topics. This implies that a word or a document can have some degree of association with every topic, though often it will be much more strongly associated with one or a few topics.

On the other hand, Latent Dirichlet Allocation (LDA) is a probabilistic topic model that represents documents as mixtures of topics and topics as mixtures of words. In LDA, each document is represented as a distribution over topics, and each topic is represented as a distribution over words. This does indeed allow for each document to be associated with multiple topics. However, it's important to note that LDA is not multi-topic "by default" for each word. Each word in each document is assigned to one topic in LDA, but the same word in a different document or different location can be assigned to a different topic. This allows for a more nuanced understanding of how words are used in different contexts.

So, in summary, while LSI/NMF and LDA both represent documents and words as mixtures of topics, the nature of these mixtures is different due to the different assumptions and mathematical techniques used by these methods. LSI/NMF allows for a word or a document to be associated with every topic to some degree, while LDA represents each document as a probabilistic mixture of topics, with each word in each document assigned to a specific topic.

LSI, which is based on Singular Value Decomposition (SVD), is usually faster than LDA for smaller to moderately sized datasets. This is because SVD is a well-studied linear algebra problem for which highly optimized solutions exist. However, LSI does not scale well with the size of the dataset because the time complexity of SVD is cubic in the number of documents and terms.

On the other hand, LDA, which is based on probabilistic graphical models, is typically more computationally intensive than LSI, especially for larger numbers of topics. However, LDA can be more easily parallelized than SVD, which means it can be more efficient than LSI for very large datasets, especially when using multiple cores or distributed computing resources.

In addition, it's worth mentioning that LDA generally provides a richer and more interpretable topic structure than LSI, especially for larger and more complex datasets. LSI tends to work better for tasks that require capturing semantic similarity, like information retrieval, while LDA is often preferred for tasks that require understanding the thematic structure of the data, like document classification or content analysis.


**Subtopic Modeling**

Subtopic modeling extends the capabilities of basic topic modeling by digging deeper into the layered structure of the topics. It identifies not only the overarching themes but also the subthemes or subtopics nested within the larger topics. This creates a more nuanced, detailed, and complete representation of the data, allowing for a greater depth of understanding.

Watson NLP, has incorporated advanced models like Hierarchical Clustering, which can perform both topic and subtopic modeling. These sophisticated models parse through the text data, identifying major topics and then breaking them down into their constituent subtopics. This allows for a more comprehensive exploration of the thematic landscape.





In [34]:
from gensim import corpora, models, similarities

# sample corpus
corpus = [
    "The fox jumps over the lazy dog",
    "I prefer reading over playing sports",
    "John and I play football every evening",
    "Reading books is a good habit",
    "I love playing football",
    "The quick brown fox jumps over the lazy dog",
]

# create a list of words for each document
texts = [[word for word in document.lower().split()] for document in corpus]
texts

[['the', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog'],
 ['i', 'prefer', 'reading', 'over', 'playing', 'sports'],
 ['john', 'and', 'i', 'play', 'football', 'every', 'evening'],
 ['reading', 'books', 'is', 'a', 'good', 'habit'],
 ['i', 'love', 'playing', 'football'],
 ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']]

In [35]:
# create a dictionary from the texts, which assigns an id to each word
dictionary = corpora.Dictionary(texts)

# convert the corpus into a bag of words representation
corpus_gensim = [dictionary.doc2bow(text) for text in texts]
corpus_gensim

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2)],
 [(4, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)],
 [(6, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)],
 [(9, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)],
 [(6, 1), (7, 1), (14, 1), (22, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (23, 1), (24, 1)]]

In [36]:
# print the word to index mapping
print(dictionary.token2id)

{'dog': 0, 'fox': 1, 'jumps': 2, 'lazy': 3, 'over': 4, 'the': 5, 'i': 6, 'playing': 7, 'prefer': 8, 'reading': 9, 'sports': 10, 'and': 11, 'evening': 12, 'every': 13, 'football': 14, 'john': 15, 'play': 16, 'a': 17, 'books': 18, 'good': 19, 'habit': 20, 'is': 21, 'love': 22, 'brown': 23, 'quick': 24}


In [29]:
# create a tfidf model to weight terms
tfidf = models.TfidfModel(corpus_gensim)

# apply tfidf to the corpus
corpus_tfidf = tfidf[corpus_gensim]



Topic 0: (0, '0.648*"the" + 0.324*"lazy" + 0.324*"jumps" + 0.324*"dog"')
Topic 1: (1, '0.437*"playing" + 0.421*"love" + 0.357*"football" + 0.338*"i"')


In [37]:
import numpy as np
for doc in corpus_tfidf:
    print([[dictionary[id], np.round(freq, decimals=2)] for id, freq in doc])

[['dog', 0.35], ['fox', 0.35], ['jumps', 0.35], ['lazy', 0.35], ['over', 0.22], ['the', 0.69]]
[['over', 0.22], ['i', 0.22], ['playing', 0.35], ['prefer', 0.57], ['reading', 0.35], ['sports', 0.57]]
[['i', 0.16], ['and', 0.43], ['evening', 0.43], ['every', 0.43], ['football', 0.26], ['john', 0.43], ['play', 0.43]]
[['reading', 0.26], ['a', 0.43], ['books', 0.43], ['good', 0.43], ['habit', 0.43], ['is', 0.43]]
[['i', 0.28], ['playing', 0.44], ['football', 0.44], ['love', 0.73]]
[['dog', 0.27], ['fox', 0.27], ['jumps', 0.27], ['lazy', 0.27], ['over', 0.17], ['the', 0.54], ['brown', 0.44], ['quick', 0.44]]


In [38]:
# train an LSI model using the tfidf-weighted corpus
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)

# print the topics
for i, topic in enumerate(lsi.print_topics(num_topics=2, num_words=4)):
    print(f"Topic {i}: {topic}")

Topic 0: (0, '0.648*"the" + 0.324*"jumps" + 0.324*"lazy" + 0.324*"dog"')
Topic 1: (1, '0.437*"playing" + 0.421*"love" + 0.357*"football" + 0.338*"i"')


In [39]:
# get LSI topic distribution for each document in the corpus
corpus_lsi = lsi[corpus_tfidf]

# print the topic distribution for each document
for i, doc in enumerate(corpus_lsi):
    print(f"Document {i}: {doc}")


Document 0: [(0, 0.9419395152303178), (1, -0.05536089245512543)]
Document 1: [(0, 0.11438430141292252), (1, 0.6612698322250348)]
Document 2: [(0, 0.01233944779540223), (1, 0.4922101112718225)]
Document 3: [(0, 0.013480280087970737), (1, 0.20646977376604278)]
Document 4: [(0, 0.034228764091513125), (1, 0.753813439479623)]
Document 5: [(0, 0.9411752486699287), (1, -0.061785782643791816)]
